In [1]:
%matplotlib notebook
from ipywidgets import *
import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider
import matplotlib
import cv2
from matplotlib.animation import FuncAnimation
import skvideo.io
import pandas as pd
import time
from datetime import timedelta
from matplotlib import gridspec

In [2]:
#Stress data
#participant C100
source='.\Stress.csv'
data=pd.read_csv(source,header=0)
l=len(data)
for line in range(1,l):
    data=data.append({'Unix timestamp': data['Unix timestamp'][line]-1, 'Stress response': data['Stress response'][line-1]}, ignore_index=True)
data=data.sort_values(by='Unix timestamp')
data.index=pd.to_datetime(data['Unix timestamp'],unit='s')

In [3]:
EDA_file=pd.read_csv(r'E:\Stress on the road\C100\EDA\1532049008_A00E6B_C100\EDA.csv',header=None)
EDAstart=EDA_file.values[0]
EDAfreq=EDA_file.values[1]
EDA=EDA_file.iloc[2:]
base=pd.to_datetime(EDAstart,unit='s')
arr = np.array([base + timedelta(milliseconds=(i*250)) for i in range(0,len(EDA))])
arr=arr.reshape(-1,)
EDA.index=arr
EDA.columns=['EDA']

In [4]:
EDA_HR_file=pd.read_csv(r'E:\Stress on the road\C100\EDA\1532049008_A00E6B_C100\HR.csv',header=None)
EDAstart_HR=EDA_file.values[0]
EDAfreq_HR=EDA_file.values[1]
EDA_HR=EDA_HR_file.iloc[2:]
base=pd.to_datetime(EDAstart_HR,unit='s')
arr = np.array([base + timedelta(seconds=(i)) for i in range(0,len(EDA_HR))])
arr=arr.reshape(-1,)
EDA_HR.index=arr
EDA_HR.columns=['HR']

In [5]:
#HR and BR data
HRsource='E:\Stress on the road\C100\Zephyr\2018_07_19-18_08_48\2018_07_19-18_08_48_General.csv'
HRdata=pd.read_csv(r'E:\Stress on the road\C100\Zephyr\2018_07_19-18_08_48\2018_07_19-18_08_48_General.csv',header=0)
HRdata.index=pd.to_datetime(HRdata['Timestamp'],dayfirst=True)
#localizing time format, convert it and delocalize it
HRdata.index=HRdata.index.tz_localize('US/Pacific').tz_convert('GMT').tz_localize(None)

In [6]:
delta = timedelta(
...     days=0,
...     seconds=5,
...     microseconds=0,
...     milliseconds=0,
...     minutes=0,
...     hours=0,
...     weeks=0
... )

In [7]:
#inF=skvideo.io.vread(r'E:\Stress on the road\C100\C100.mov')

In [8]:
Vidstart=1532049554.04
Vidstart_time=pd.to_datetime('1532049554.04',unit='s')
fps=20

In [15]:
help(cv2.cuda)

Help on module cv2.cuda in cv2:

NAME
    cv2.cuda

FUNCTIONS
    BroxOpticalFlow_create(...)
        BroxOpticalFlow_create([, alpha[, gamma[, scale_factor[, inner_iterations[, outer_iterations[, solver_iterations]]]]]]) -> retval
        .
    
    CascadeClassifier_create(...)
        CascadeClassifier_create(filename) -> retval
        .   @brief Loads the classifier from a file. Cascade type is detected automatically by constructor parameter.
        .   
        .   @param filename Name of the file from which the classifier is loaded. Only the old haar classifier
        .   (trained by the haar training application) and NVIDIA's nvbin are supported for HAAR and only new
        .   type of OpenCV XML cascade supported for LBP. The working haar models can be found at opencv_folder/data/haarcascades_cuda/
    
    DensePyrLKOpticalFlow_create(...)
        DensePyrLKOpticalFlow_create([, winSize[, maxLevel[, iters[, useInitialFlow]]]]) -> retval
        .
    
    DescriptorMatcher

In [14]:
def grab_frame(cap):
    ret,frame = cap.read()
    return cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
def update_image(j):
    cap.set(1,(slider2.value*fps)+j)
    im2.set_data(grab_frame(cap))
    
fps=20
xmin=max(min(HRdata.index),Vidstart_time)
xmax=max(HRdata.index)
x1=xmin
x2=x1+delta
frame=5*fps
vid_start=((xmin-Vidstart_time).seconds)*fps
expstart=Vidstart_time+delta
cap = cv2.VideoCapture(r'E:\FFOutput\C100~2.mov')
cap.set(1,vid_start)

max_val=(max(HRdata.index)-min(HRdata.index)).seconds
#create two subplots
fig = plt.figure(figsize=(8,8))
gs  = gridspec.GridSpec(4, 1, height_ratios=[4, 1 ,1, 1])
ax2 = plt.subplot(gs[0])
ax1 = plt.subplot(gs[3])
ax3 = plt.subplot(gs[2])
ax4 = plt.subplot(gs[1])

#ax1=fig.add_subplot(3,1,3)
ax1.plot(HRdata.index,HRdata.HR,'b',linestyle='-',label='HR')
ax1.plot(HRdata.index,HRdata.BR,'r',linestyle='-',label='BR')
ax3.plot(data.index,data['Stress response'],marker='x',markersize=12,label='Stressor')
ax4.plot(EDA.index,EDA.EDA,label='EDA')
#ax1.axvline(linewidth=4, color='r')
ax1.axvline(x=expstart,linewidth=4,color='r')
ax4.axvline(x=expstart,linewidth=4,color='r')
ax3.axvline(x=expstart,linewidth=4,color='r')
ax2.axis('off')
#create two image plots
#im1 = ax1.imshow(grab_frame(cap1))
im2 = ax2.imshow(grab_frame(cap))
ax1.legend()
ax1.set_title('Zephyr HR,BR and EDA HR')
ax3.set_title('Stress response')  
ax4.set_title('EDA')  
ax1.get_shared_x_axes().join(ax1, ax3,ax4)
#ax1.get_shared_x_axes().join(ax1, ax4)
ax1.set_xlim(0,10)
ax3.axes.get_xaxis().set_visible(False)
ax4.axes.get_xaxis().set_visible(False)

#file=open('FigureObject.fig.pickle', 'wb')
#pickle.dump(fig, file)
#file.close
plt.show()




"""
def update_image(j):
    #print(j)
    global x1,x2
    #if j<(slider3.value*fps):
        #print("Updaing ",j)
    im2 = ax2.imshow(inF[vid_start+((x1-xmin).seconds*fps)+j])
"""
ani1 = FuncAnimation(plt.gcf(), update_image, frames=15*20, interval=20/20,repeat=True)



def update_axis(w,s):
    global xmin,xmax,x1,x2,frame_rate,frame
    delta_w=timedelta(seconds=w)
    delta_s=timedelta(seconds=s)
    x1=xmin+delta_w
    x2=min(xmin+delta_w+delta_s,xmax)
    ax1.set_xlim(x1,x2)
    fig.canvas.draw_idle()
    #ani1.event_source.stop()

slider3=IntSlider(min=1, max=max_val, step=1, value=5,description='Zoom')
slider2=IntSlider(min=0, max=max_val, step=1, value=0,description='Pan')
interact(update_axis, w=slider2,s=slider3);    




from IPython.display import HTML
#HTML(ani.to_jshtml())
#HTML(ani1.to_jshtml())
#ani.save('testing.mp4')

AttributeError: module 'cv2.cuda' has no attribute 'VideoReader'

In [21]:
help(cv2.cvtColor)

Help on built-in function cvtColor:

cvtColor(...)
    cvtColor(src, code[, dst[, dstCn]]) -> dst
    .   @brief Converts an image from one color space to another.
    .   
    .   The function converts an input image from one color space to another. In case of a transformation
    .   to-from RGB color space, the order of the channels should be specified explicitly (RGB or BGR). Note
    .   that the default color format in OpenCV is often referred to as RGB but it is actually BGR (the
    .   bytes are reversed). So the first byte in a standard (24-bit) color image will be an 8-bit Blue
    .   component, the second byte will be Green, and the third byte will be Red. The fourth, fifth, and
    .   sixth bytes would then be the second pixel (Blue, then Green, then Red), and so on.
    .   
    .   The conventional ranges for R, G, and B channel values are:
    .   -   0 to 255 for CV_8U images
    .   -   0 to 65535 for CV_16U images
    .   -   0 to 1 for CV_32F images
    .   
    .

In [24]:
import cv2
from UMatFileVideoStream import UMatFileVideoStream

video = UMatFileVideoStream('E:\FFOutput\C100~2.mov').start()
rgb = cv2.UMat(video.height, video.width, cv2.CV_8UC3)
while not video.stopped:
    hsv = cv2.cvtColor(video.read(), cv2.COLOR_BGR2RGB)
    # more of processing before fetching the images
    hsv = cv2.cvtColor(hsv, cv2.COLOR_HSV2RGB)
    img = hsv.get()   # image is now a numpy array

KeyboardInterrupt: 

In [25]:
(HRdata.index-xmin).seconds

Int64Index([85774, 85775, 85776, 85777, 85778, 85779, 85780, 85781, 85782,
            85783,
            ...
             4818,  4819,  4820,  4821,  4822,  4823,  4824,  4825,  4826,
             4827],
           dtype='int64', name='Timestamp', length=5454)

In [ ]:
fig,(ax2)=plt.subplots(1,1,figsize=(12,12))
im2=ax2.imshow(inF[0])
plt.show()
def update_image(j):
    #print(j)
    global x1,x2
    #if j<(slider3.value*fps):
        #print("Updaing ",j)
    im2=ax2.imshow(inF[(j)])

ani1 = FuncAnimation(fig, update_image, frames=(5*20), interval=1000/20,repeat=True)

In [ ]:
cap = cv2.VideoCapture(r'E:\Stress on the road\Pilots\testforpablo.mov')  

def grab_frame(cap):
    ret,frame = cap.read()
    return cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)

def update_image(j):
    im2.set_data(grab_frame(cap))
    
  
fig1,(ax2)=plt.subplots(1,1,figsize=(12,12))
im2 = ax2.imshow(grab_frame(cap))
plt.show()
ani1 = FuncAnimation(plt.gcf(), update_image, frames=5*20, interval=300/20,repeat=False)
    